## Import Library

In [1]:
import zipfile
from bs4 import BeautifulSoup
import unicodedata
import re
import pandas as pd
import numpy as np

## Open and read docx

In [2]:
#open docx with zipfile
doc_zip = zipfile.ZipFile('FORM A.docx')
# doc_zip.namelist()

In [3]:
#read xml
doc_xml = doc_zip.read('word/document.xml')

In [4]:
# parse xml 
soup_xml = BeautifulSoup(doc_xml, 'xml')
# find body
body = soup_xml.find('body')

In [5]:
# print all text
for i, p in enumerate(body.find_all(['w:p','w:br'])):
    string = unicodedata.normalize('NFKD', p.get_text())
    # print(i,p.get_text())

In [6]:
last_line_nbr = len(body.find_all(['w:p','w:br']))-1

## Extract Questions

In [7]:
# construct empty dictionary 'questions'
questions = {'question_line_nbr':[],
      'question':[]}

for i, p in enumerate(body.find_all('w:p')):
    string = unicodedata.normalize('NFKD', p.get_text())

    # if first 4 charactors in the paragraph has 'number dot space' save it to questions dict.
    if bool(re.search('\d+\. ', string[:4])):
        # print(i,string)
        questions['question_line_nbr'].append(i)
        questions['question'].append(string)

In [8]:
# append the last line number to the questions dict.- it is needed for combining question and answer.
questions['question_line_nbr'].append(last_line_nbr)
questions['question'].append(np.nan)

In [9]:
# convert to Dataframe
df_q = pd.DataFrame(questions)

## Extract Answers

### Get Answer A, B, C, and D,  and save them to Dictionary Answers 1, 2, 3, and 4

In [10]:
# construct empty dictionary 'answers1'
answers1 = {'answer1_line_nbr':[],
      'answer1':[]}

for i, p in enumerate(body.find_all('w:p')):
    string = unicodedata.normalize('NFKD', p.get_text())
    
    # if first 4 charactors in the paragraph has 'A dot space' save it to answers1 dict.
    if bool(re.search('A\. ', string[:4])):
        # print(i,string)
        answers1['answer1_line_nbr'].append(i)
        answers1['answer1'].append(string)

In [11]:
# construct empty dictionary 'answers2'
answers2 = {'answer2_line_nbr':[],
      'answer2':[]}

for i, p in enumerate(body.find_all('w:p')):
    string = unicodedata.normalize('NFKD', p.get_text())
    
    # if first 4 charactors in the paragraph has 'B dot space' save it to answers2 dict.
    if bool(re.search('B\. ', string[:4])):
        # print(i,string)
        answers2['answer2_line_nbr'].append(i)
        answers2['answer2'].append(string)

In [12]:
# construct empty dictionary 'answers3'
answers3 = {'answer3_line_nbr':[],
      'answer3':[]}

for i, p in enumerate(body.find_all('w:p')):
    string = unicodedata.normalize('NFKD', p.get_text())
    
    # if first 4 charactors in the paragraph has 'C dot space' save it to answers3 dict.
    if bool(re.search('C\. ', string[:4])):
        # print(i,string)
        answers3['answer3_line_nbr'].append(i)
        answers3['answer3'].append(string)

In [13]:
# construct empty dictionary 'answers4'
answers4 = {'answer4_line_nbr':[],
      'answer4':[]}

for i, p in enumerate(body.find_all('w:p')):
    string = unicodedata.normalize('NFKD', p.get_text())
    
    # if first 4 charactors in the paragraph has 'D dot space' save it to answers4 dict.
    if bool(re.search('D\. ', string[:4])):
        # print(i,string)
        answers4['answer4_line_nbr'].append(i)
        answers4['answer4'].append(string)

In [14]:
# convert to Dataframe
df_a1 = pd.DataFrame(answers1)
df_a2 = pd.DataFrame(answers2)
df_a3 = pd.DataFrame(answers3)
df_a4 = pd.DataFrame(answers4)

## Combine Questions and Answers

In [15]:
df_a1[df_a1['answer1_line_nbr'].between(20,31)]['answer1_line_nbr'].tolist()[0]

26

In [16]:
# find line number of answer between line number of questions
for i in range(len(df_q)-1):
    start=df_q['question_line_nbr'][i]
    end=df_q['question_line_nbr'][i+1]
    
    
    # return pd series containing answer1, and answer1_line_number (Original answer A) for question 
    answer1_ln_nbr = df_a1[df_a1['answer1_line_nbr'].between(start,end)]['answer1_line_nbr']
    answer1 = df_a1[df_a1['answer1_line_nbr'].between(start,end)]['answer1']
    
    # return pd series containing answer2, and answer2_line_number (Original answer B) for question    
    answer2_ln_nbr = df_a2[df_a2['answer2_line_nbr'].between(start,end)]['answer2_line_nbr']
    answer2 = df_a2[df_a2['answer2_line_nbr'].between(start,end)]['answer2']
    
    # return pd series containing answer3, and answer3_line_number (Original answer C) for question    
    answer3_ln_nbr = df_a3[df_a3['answer3_line_nbr'].between(start,end)]['answer3_line_nbr']
    answer3 = df_a3[df_a3['answer3_line_nbr'].between(start,end)]['answer3']

    # return pd series containing answer4, and answer4_line_number (Original answer D) for question    
    answer4_ln_nbr = df_a4[df_a4['answer4_line_nbr'].between(start,end)]['answer4_line_nbr']
    answer4 = df_a4[df_a4['answer4_line_nbr'].between(start,end)]['answer4']

    
    # if there is answer1 line number and answer1 for the question append to line number and answer column
    if answer1_ln_nbr.empty==False:
        df_q.loc[i,'answer1_line_nbr']=answer1_ln_nbr.tolist()[0]
    if answer1.empty==False:
        df_q.loc[i,'answer1']=answer1.tolist()[0]       
        
    # if there is answer2 line number and answer2 for the question append to line number and answer column        
    if answer2_ln_nbr.empty==False:
        df_q.loc[i,'answer2_line_nbr']=answer2_ln_nbr.tolist()[0]        
    if answer2.empty==False:
        df_q.loc[i,'answer2']=answer2.tolist()[0]
        
    # if there is answer3 line number and answer3 for the question append to line number and answer column        
    if answer3_ln_nbr.empty==False:
        df_q.loc[i,'answer3_line_nbr']=answer3_ln_nbr.tolist()[0]    
    if answer3.empty==False:
        df_q.loc[i,'answer3']=answer3.tolist()[0]
       
    # if there is answer4 line number and answer4 for the question append to line number and answer column        
    if answer4_ln_nbr.empty==False:
        df_q.loc[i,'answer4_line_nbr']=answer4_ln_nbr.tolist()[0]       
    if answer4.empty==False:
        df_q.loc[i,'answer4']=answer4.tolist()[0]

## Extract Keys

In [17]:
# construct empty dictionary 'keys'
keys = {'key':[]}

for i, p in enumerate(body.find_all('w:p')):
    string = unicodedata.normalize('NFKD', p.get_text())

    # if first 10 charactors in the paragraph has 'FORM A KEY' save it to keys dict.
    if bool(re.search('FORM A KEY', string[:10])):
        key_ln = i+1

In [18]:
for i in range(key_ln,last_line_nbr):
    string = unicodedata.normalize('NFKD', body.find_all('w:p')[i].get_text())
    keys['key'].append(string)

In [19]:
df_keys = pd.DataFrame(keys)

In [20]:
df_q['keys']=df_keys['key']

## Exracted Data

In [21]:
df_q

,question_line_nbr,question,answer1_line_nbr,answer1,answer2_line_nbr,answer2,answer3_line_nbr,answer3,answer4_line_nbr,answer4,keys
0,20,1. A list of four physical quantities is,26.0,A. 1,27.0,B. 2,28.0,C. 3,29.0,D. 4,C
1,31,2. The uncertainty in reading a laboratory th...,32.0,A. 2.5 %,33.0,B. 5 %,34.0,C. 7.5 %,35.0,D. 10 %,D
2,36,3. An object of mass m is hung from a spring. ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B
3,42,"4. Two different experiments, P and Q, generat...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C
4,47,5. The road from city X to city Y is 1000 km ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D
5,52,6. An object is thrown upwards leaving the th...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A
6,54,7. A car accelerates uniformly from rest to a...,56.0,A. v t2,57.0,B. 12vt2-t1+v t1,58.0,C. 12vt2+t1,59.0,D. 12v t1+vt2-t1,D
7,60,8. The graph below shows how velocity v varie...,63.0,A. displacement.,64.0,B. final velocity.,65.0,C. change in velocity.,66.0,D. acceleration.,A
8,68,9. A boy throws a ball horizontally at a spee...,70.0,A. 40 m,71.0,B. 60 m,72.0,C. 80 m,73.0,D. 320 m,C
9,75,10. A rocket has just been launched vertically...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B


In [22]:
df_q.to_csv('FormA.csv',index=False)

In [23]:
df_to_shuffle = df_q.dropna()

In [24]:
df_to_shuffle.to_csv('Questions_to_shuffle.csv',index=False)